# 00 Load Data To Pickle

Author: Harry Yau

Date: Aug 28, 2019

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import glob as glob
import pickle

In [2]:
data_folder_mobi = 'data/Mobi'
data_folder_weather = 'data/weather'
pickle_folder = 'pickle'

Load Excel and CSV files into pickle format. The data was retrieved from https://www.mobibikes.ca/en/system-data. On the date of data retrieval, only data up to March 2019 was available.

In [3]:
xlsx_filelist = glob.glob(data_folder_mobi + '/*.xlsx')
csv_filelist = glob.glob(data_folder_mobi + '/*.csv')

data_list=[]
for file in xlsx_filelist:
    print(file)
    data_list.append(pd.read_excel(file))
    
#2019, the files became a csv file.
for file in csv_filelist:
    print(file)
    data_list.append(pd.read_csv(file))

data/Mobi\Mobi_System_Data_2017.xlsx
data/Mobi\Mobi_System_Data_2018-01.xlsx
data/Mobi\Mobi_System_Data_2018-02.xlsx
data/Mobi\Mobi_System_Data_2018-03.xlsx
data/Mobi\Mobi_System_Data_2018-04.xlsx
data/Mobi\Mobi_System_Data_2018-05.xlsx
data/Mobi\Mobi_System_Data_2018-06.xlsx
data/Mobi\Mobi_System_Data_2018-07.xlsx
data/Mobi\Mobi_System_Data_2018-08.xlsx
data/Mobi\Mobi_System_Data_2018-09.xlsx
data/Mobi\Mobi_System_Data_2018-10.xlsx
data/Mobi\Mobi_System_Data_2018-11.xlsx
data/Mobi\Mobi_System_Data_2018-12.csv
data/Mobi\Mobi_System_Data_2019-01.csv


C:\Users\yauha\Anaconda3\envs\timeseries\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


data/Mobi\Mobi_System_Data_2019-02.csv
data/Mobi\Mobi_System_Data_2019-03.csv


In [4]:
filename = 'loaded_mobi_data.pkl'
outfile = open(pickle_folder + '/' + filename,'wb')
pickle.dump(data_list, outfile)
outfile.close()

Load Meteorological files into pickle format using functions from https://github.com/vrif/envistaweb-decoder. Specifically, the function `load_AQ_date_excel()` was used. The weather station that was used was 'Vancouver International Airport #2'. The time format of the data is in timezone PST (no daylight savings), as per the BC Air Data Archive website. 

In [5]:
def load_AQ_data_excel(url):
    
    '''
    Imports Hourly XLS files from https://envistaweb.env.gov.bc.ca/DynamicTable2.aspx?G_ID=331
    Please see readme on how to download files from the website.
    This decoder only works with hourly data.
    '''
    
    data = pd.read_excel(url)
    
    data.columns = data.loc[0]
    data.drop(index=[0,1], inplace=True)
    data.drop(index=data[-8:].index, inplace=True) #Removes bottom rows.
    
    for i in range(data.shape[0]):
        if (data['Time'].iloc[i] == '24:00 AM'):
            data['Date'].iloc[i] += pd.DateOffset(1)
        elif (isinstance(data['Time'].iloc[i], datetime.time)):
            data['Date'].iloc[i] += pd.to_timedelta(data['Time'].iloc[i].strftime("%H:%M:%S"))
        else:
            pass
    
    data.drop(columns='Time', inplace=True)
    data['Date'] = pd.to_datetime(data['Date']) #converts to datetime
    data.reset_index(drop=True, inplace=True)
    
    return(data)

In [9]:
file_list = glob.glob(data_folder_weather + '/*.xls')

combined_df = []

for i, file in enumerate(file_list):
    df = load_AQ_data_excel(file)
    combined_df.append(df)
    print(file)

combined_df = pd.concat(combined_df, axis=0)

filename = 'YVR_wxdata.pkl'
outfile = open(pickle_folder + '/' + filename,'wb')
pickle.dump(combined_df, outfile)
outfile.close()

data/weather\yvr_2017.xls
data/weather\yvr_2018.xls
data/weather\yvr_2019.xls
